<a href="https://colab.research.google.com/github/mralamdari/Computer-Vision-Papers/blob/main/AlexNet_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import tensorflow
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#Data 
[Source](https://www.kaggle.com/datasets/csafrit2/plant-leaves-for-image-classification)

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive'
!kaggle datasets download -d csafrit2/plant-leaves-for-image-classification
!unzip \*.zip && rm .zip

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory  = '/content/Plants_2/train/',
    label_mode = 'categorical',
    color_mode = 'rgb',
    batch_size = 32,
    image_size = (224, 224),
    shuffle    = True
)


valid_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory  = '/content/Plants_2/valid/',
    label_mode = 'categorical',
    color_mode = 'rgb',
    batch_size = 32,
    image_size = (224, 224),
    shuffle    = True
)


test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory  = '/content/Plants_2/test/',
    label_mode = 'categorical',
    color_mode = 'rgb',
    batch_size = 32,
    image_size = (224, 224),
    shuffle    = True
)